In [1]:
import os

In [2]:
%pwd

'/media/kalema/9954-79C8/Projects/Swahili-News-Classifier/research'

In [3]:
os.chdir('../')


In [4]:
%pwd

'/media/kalema/9954-79C8/Projects/Swahili-News-Classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

from swahiliNewsClassifier.constants import *
from swahiliNewsClassifier.utilities.helper_functions import read_yaml, create_directories

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    train_source_URL: str
    test_source_URL: str
    train_data_file: Path
    test_data_file: Path

    unzip_dir: Path

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        """
        Initialize ConfigurationManager with configuration and parameter files.

        Args:
            config_filepath (str): Path to the configuration YAML file.
            params_filepath (str): Path to the parameters YAML file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Get the data ingestion configuration.

        Returns:
            DataIngestionConfig: Configuration object for data ingestion.
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=config.root_dir,
            train_source_URL=config.train_source_URL,
            test_source_URL=config.test_source_URL,
            train_data_file=config.train_data_file,
            test_data_file=config.test_data_file,
            unzip_dir=config.unzip_dir
        )


In [8]:
import os
import zipfile
import gdown
from swahiliNewsClassifier import classifierlogger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initialize DataIngestion object with the provided configuration.

        Args:
            config (DataIngestionConfig): Configuration object for data ingestion.
        """
        self.config = config

    def download_file(self):
        """Fetch data from a URL.
        
        Raises:
            Exception: If an error occurs during the download process.
        """
        os.makedirs("artifacts/data_ingestion", exist_ok=True)
        dataset_urls = [self.config.train_source_URL, self.config.test_source_URL]
        zip_download_dir = [self.config.train_data_file, self.config.test_data_file]
        
        for url, dest in zip(dataset_urls, zip_download_dir):
            try:
                classifierlogger.info(f"Downloading data from {url} into file {dest}")

                file_id = url.split("/")[-2]
                prefix = "https://drive.google.com/uc?/export=download&id="
                gdown.download(prefix + file_id, dest)

                classifierlogger.info(f"Downloaded data from {url} into file {dest}")
            except Exception as e:
                classifierlogger.error(f"Error downloading file from {url} to {dest}")
                raise e

    def extract_zip_file(self):
        """Extract a zip file.

        This method extracts the contents of a zip file specified in the configuration
        to the directory specified in the configuration.

        Raises:
            Exception: If an error occurs during the extraction process.
        """
        zip_download_dir = [self.config.train_data_file, self.config.test_data_file]
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        for zip_file in zip_download_dir:
            try:
                with zipfile.ZipFile(zip_file, "r") as zip_ref:
                    zip_ref.extractall(unzip_path)

                classifierlogger.info(f"Extracted zip file {zip_file} into: {unzip_path}")
            except Exception as e:
                classifierlogger.error(f"Error extracting zip file: {zip_file}")
                raise e


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-06-08 15:24:19,735: INFO: helper_functions:20: yaml file: configuration/configuration.yaml loaded successfully]
[2024-06-08 15:24:19,744: INFO: helper_functions:20: yaml file: parameters.yaml loaded successfully]
[2024-06-08 15:24:19,747: INFO: helper_functions:35: Created directory at: artifacts]
[2024-06-08 15:24:19,749: INFO: helper_functions:35: Created directory at: artifacts/data_ingestion]
[2024-06-08 15:24:19,750: INFO: 2052182238:23: Downloading data from https://drive.google.com/file/d/15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB/view?usp=sharing into file artifacts/data_ingestion/train_data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB
To: /media/kalema/9954-79C8/Projects/Swahili-News-Classifier/artifacts/data_ingestion/train_data.zip
100%|██████████| 3.78M/3.78M [00:03<00:00, 1.12MB/s]

[2024-06-08 15:24:27,176: INFO: 2052182238:29: Downloaded data from https://drive.google.com/file/d/15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB/view?usp=sharing into file artifacts/data_ingestion/train_data.zip]
[2024-06-08 15:24:27,178: INFO: 2052182238:23: Downloading data from https://drive.google.com/file/d/1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er/view?usp=sharing into file artifacts/data_ingestion/test_data.zip]



Downloading...
From: https://drive.google.com/uc?/export=download&id=1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er
To: /media/kalema/9954-79C8/Projects/Swahili-News-Classifier/artifacts/data_ingestion/test_data.zip
100%|██████████| 992k/992k [00:00<00:00, 1.19MB/s]

[2024-06-08 15:24:30,890: INFO: 2052182238:29: Downloaded data from https://drive.google.com/file/d/1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er/view?usp=sharing into file artifacts/data_ingestion/test_data.zip]


[2024-06-08 15:24:31,090: INFO: 2052182238:52: Extracted zip file artifacts/data_ingestion/train_data.zip into: artifacts/data_ingestion]
[2024-06-08 15:24:31,150: INFO: 2052182238:52: Extracted zip file artifacts/data_ingestion/test_data.zip into: artifacts/data_ingestion]
